In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import os
base_path = '../../'
sys.path.append(base_path)

In [23]:
from src.dgl_flow_field_dataset import DGLSurfaceFlowFieldDataset
from src.pyvista_flow_field_dataset import PyvistaFlowFieldDataset
ds_pv = PyvistaFlowFieldDataset.load_from_huggingface("datasets/pyvista",num_samples=50)
#ds_dgl = DGLSurfaceFlowFieldDataset(os.path.join(base_path,'datasets/dgl_surface'),ds_pv)
ds_dgl = DGLSurfaceFlowFieldDataset(os.path.join(base_path,'datasets/dgl_surface'))
ds_dgl[0]

Loaded 50 samples from 'datasets/pyvista'.


Graph(num_nodes=36577, num_edges=142478,
      ndata_schemes={'BodyID': Scheme(shape=(), dtype=torch.int32), 'CellArea': Scheme(shape=(), dtype=torch.float32), 'Normal': Scheme(shape=(3,), dtype=torch.float32), 'ShearStress': Scheme(shape=(3,), dtype=torch.float32), 'Position': Scheme(shape=(3,), dtype=torch.float32), 'Temperature': Scheme(shape=(), dtype=torch.float32), 'Pressure': Scheme(shape=(), dtype=torch.float32)}
      edata_schemes={'dx': Scheme(shape=(3,), dtype=torch.float32)})

In [4]:
num_train_samples = int(len(ds_dgl) * 0.8)
num_val_samples = len(ds_dgl) - num_train_samples
ds_dgl.shuffle()
train_ds = ds_dgl.slice(0, num_train_samples)
val_ds = ds_dgl.slice(num_train_samples, num_train_samples + num_val_samples)

In [5]:
ds_pv[1].surface_data[0][1]

UnstructuredGrid (0x707f369b7dc0)
  N Cells:    1002
  N Points:   1042
  X Bounds:   8.316e-02, 1.167e-01
  Y Bounds:   9.825e-03, 4.827e-02
  Z Bounds:   0.000e+00, 1.500e-02
  N Arrays:   12

In [6]:
ds_pv[0].get_surface_points(block_index=0)

pyvista_ndarray([[0.498     , 0.        , 0.        ],
                 [0.5       , 0.002     , 0.        ],
                 [0.496     , 0.        , 0.        ],
                 ...,
                 [0.47394846, 0.08794846, 0.        ],
                 [0.474     , 0.058     , 0.02      ],
                 [0.47194449, 0.08594449, 0.        ]])

In [7]:
ds_pv[1].plot_volume("Pressure")

2025-05-26 13:34:12.926 (   0.710s) [    7080E3A17740]      vtkCGNSReader.cxx:4268  WARN| vtkCGNSReader (0x64a44e732780): Skipping BC_t node: BC_t type 'BCInflow' not supported yet.
2025-05-26 13:34:12.926 (   0.710s) [    7080E3A17740]      vtkCGNSReader.cxx:4268  WARN| vtkCGNSReader (0x64a44e732780): Skipping BC_t node: BC_t type 'BCSymmetryPlane' not supported yet.
2025-05-26 13:34:12.926 (   0.710s) [    7080E3A17740]      vtkCGNSReader.cxx:4268  WARN| vtkCGNSReader (0x64a44e732780): Skipping BC_t node: BC_t type 'BCTunnelOutflow' not supported yet.


Widget(value='<iframe src="http://localhost:36067/index.html?ui=P_0x707f368e5750_0&reconnect=auto" class="pyvi…

In [8]:
ds_dgl[0]

Graph(num_nodes=35771, num_edges=139880,
      ndata_schemes={'BodyID': Scheme(shape=(), dtype=torch.int32), 'CellArea': Scheme(shape=(), dtype=torch.float32), 'Normal': Scheme(shape=(3,), dtype=torch.float32), 'ShearStress': Scheme(shape=(3,), dtype=torch.float32), 'Position': Scheme(shape=(3,), dtype=torch.float32), 'Temperature': Scheme(shape=(), dtype=torch.float32), 'Pressure': Scheme(shape=(), dtype=torch.float32)}
      edata_schemes={'dx': Scheme(shape=(3,), dtype=torch.float32)})

In [9]:
from dgl.dataloading import GraphDataLoader
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
dataloader = GraphDataLoader(train_ds, batch_size=1, shuffle=True)
val_dataloader = GraphDataLoader(val_ds, batch_size=1, shuffle=False)

In [10]:
import dgl


def get_node_edge_X(graph: dgl.DGLGraph):
    node_X = torch.cat([graph.ndata["Position"], graph.ndata["Normal"]],dim=1)
    edge_X = torch.cat([graph.edata["dx"]],dim=1)
    return node_X, edge_X

def get_node_Y(graph: dgl.DGLGraph):
    return torch.cat([graph.ndata["Pressure"].unsqueeze(1),graph.ndata["Temperature"].unsqueeze(1),graph.ndata['ShearStress']],dim=1)

def set_graph_features(graph: dgl.DGLGraph, node_X, edge_X, node_Y):
    graph.ndata["Position"] = node_X[:,:3]
    graph.ndata["Normal"] = node_X[:,3:]
    graph.edata["dx"] = edge_X
    graph.ndata["Pressure"] = node_Y[:,0]
    graph.ndata["Temperature"] = node_Y[:,1]
    graph.ndata["ShearStress"] = node_Y[:,2:]
g=ds_dgl[0]
g_cp=g.clone()
ndx, edx = get_node_edge_X(g)
ndy = get_node_Y(g)
set_graph_features(g_cp, ndx, edx, ndy)
assert torch.allclose(g_cp.ndata["Position"], g.ndata["Position"])
assert torch.allclose(g_cp.ndata["Normal"], g.ndata["Normal"])
assert torch.allclose(g_cp.edata["dx"], g.edata["dx"])
assert torch.allclose(g_cp.ndata["Pressure"], g.ndata["Pressure"])
assert torch.allclose(g_cp.ndata["Temperature"], g.ndata["Temperature"])
assert torch.allclose(g_cp.ndata["ShearStress"], g.ndata["ShearStress"])
num_node_features = ndx.shape[1]
num_edge_features = edx.shape[1]
num_node_labels = ndy.shape[1]
print("Node X: ",ndx, ndx.shape)
print("Edge X: ",edx, edx.shape)
print("Node Y: ",ndy, ndy.shape)

Node X:  tensor([[-1.6407e+00,  1.4430e+00, -1.2733e+00,  4.5959e-03,  3.5109e-03,
         -1.1739e+00],
        [-1.6264e+00,  1.4430e+00, -1.2733e+00,  4.5959e-03,  3.5109e-03,
         -1.1739e+00],
        [-1.6407e+00,  1.3733e+00, -1.2733e+00,  4.5959e-03,  3.5109e-03,
         -1.1739e+00],
        ...,
        [ 1.6980e+00, -1.4322e+00, -1.2313e+00,  4.2439e+00,  3.5109e-03,
          1.2377e-02],
        [ 1.6980e+00, -1.3734e+00, -1.2613e+00,  4.2439e+00,  3.5109e-03,
          1.2377e-02],
        [ 1.6980e+00, -1.4322e+00, -1.2613e+00,  4.2439e+00,  3.5109e-03,
          1.2377e-02]]) torch.Size([35771, 6])
Edge X:  tensor([[ 1.5488e+00,  4.0420e-10,  2.5981e-10],
        [-3.2793e-10, -1.9337e+00,  2.5981e-10],
        [-1.5488e+00,  4.0420e-10,  2.5981e-10],
        ...,
        [-3.2793e-10, -1.6311e+00,  2.5981e-10],
        [-3.2793e-10,  4.0420e-10,  4.4761e-01],
        [-3.2793e-10,  1.6311e+00,  2.5981e-10]]) torch.Size([139880, 3])
Node Y:  tensor([[ 1.3031e-01, 

In [11]:
from modulus.models.meshgraphnet import MeshGraphNet
model = MeshGraphNet(
    input_dim_nodes=num_node_features,
    input_dim_edges=num_edge_features,
    output_dim=num_node_labels,
    aggregation='sum',
    hidden_dim_edge_encoder=64,
    hidden_dim_node_encoder=64,
    hidden_dim_processor=64,
    hidden_dim_node_decoder=64
)
model=model.to(device)
from torch.optim import Adam
from torch.optim.lr_scheduler import LambdaLR
from torch.amp import GradScaler
optimizer = Adam(model.parameters(), lr=1e-4)
scheduler = LambdaLR(optimizer, lr_lambda=lambda epoch: 0.99985 ** epoch)
scaler = GradScaler(device=device)

In [18]:

from modulus.launch.utils import save_checkpoint, load_checkpoint
checkpoint_path = 'checkpoints'
os.makedirs(checkpoint_path,exist_ok=True)
epoch_init = load_checkpoint(checkpoint_path,model,optimizer,scheduler,scaler,device=device)
num_epochs = 400
model.to(device)

def compute_loss(batch):
    batch = batch.to(device)
    node_X, edge_X = get_node_edge_X(batch)
    node_Y = get_node_Y(batch)
    node_Y_pred = model(node_X,edge_X,batch)
    batch_pred_graph = batch.clone()
    set_graph_features(batch_pred_graph, node_X, edge_X, node_Y_pred)
    agg_force_pred = ds_dgl.compute_aggregate_force(batch_pred_graph)
    agg_force = ds_dgl.compute_aggregate_force(batch)
    #print('Agg force pred: ',format_vector(agg_force_pred.tolist()),' Agg force: ',format_vector(agg_force.tolist()))
    return torch.nn.functional.mse_loss(node_Y_pred,node_Y) + 5* torch.nn.functional.mse_loss(agg_force_pred,agg_force)
for epoch in range(epoch_init,num_epochs):
    total_loss = 0
    for batch in dataloader:
        loss = compute_loss(batch)
        total_loss += loss.item()
        optimizer.zero_grad()
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
    total_loss /= len(train_ds)
    total_loss = total_loss
    val_loss = 0
    for batch in val_dataloader:
        val_loss += compute_loss(batch).item()
    val_loss /= len(val_ds)
    print(f'Epoch {epoch} loss: {total_loss} val_loss: {val_loss}')
    scheduler.step()
    if epoch % 10 == 0:
        save_checkpoint(checkpoint_path,model,optimizer,scheduler,scaler,epoch)
save_checkpoint(checkpoint_path,model,optimizer,scheduler,scaler,num_epochs)

Could not find valid model file /home/olep/Documents/flow_field_dataset/examples/meshgraphnet/checkpoints/MeshGraphNet.0.0.mdlus, skipping load
Could not find valid checkpoint file, skipping load


Epoch 0 loss: 1.046351133659482 val_loss: 0.4740852639079094
Epoch 1 loss: 0.9650482021272182 val_loss: 0.4349715456366539
Epoch 2 loss: 0.9458094319328666 val_loss: 0.46294390261173246
Epoch 3 loss: 0.939226520806551 val_loss: 0.4319621011614799
Epoch 4 loss: 0.9239674463868142 val_loss: 0.44288818538188934
Epoch 5 loss: 0.9217682892456651 val_loss: 0.42642504572868345
Epoch 6 loss: 0.930337182059884 val_loss: 0.4440990313887596
Epoch 7 loss: 0.9196593062952161 val_loss: 0.4646159097552299
Epoch 8 loss: 0.9383936071768403 val_loss: 0.4425138980150223
Epoch 9 loss: 0.9097407344728708 val_loss: 0.42522486448287966
Epoch 10 loss: 0.9167431684210896 val_loss: 0.4466161772608757
Epoch 11 loss: 0.9151038797572255 val_loss: 0.44791983515024186
Epoch 12 loss: 0.9092549813911319 val_loss: 0.4503635555505753
Epoch 13 loss: 0.9124904556199909 val_loss: 0.444341616332531
Epoch 14 loss: 0.8948028875514865 val_loss: 0.4194667488336563
Epoch 15 loss: 0.8952676793560386 val_loss: 0.44865152090787885


In [19]:
model.to(device)
g=ds_dgl[0].to(device)
g_pred = g.clone().to(device)
ndx, edx = get_node_edge_X(g)
with torch.no_grad():
    y_pred = model(ndx,edx,g)
set_graph_features(g_pred, ndx, edx, y_pred)
ds_dgl.plot_surface(g_pred,"Pressure")

Widget(value='<iframe src="http://localhost:36067/index.html?ui=P_0x707f30409610_3&reconnect=auto" class="pyvi…

In [20]:
ds_dgl.plot_surface(g,"Pressure")

Widget(value='<iframe src="http://localhost:36067/index.html?ui=P_0x707e7a5a8050_4&reconnect=auto" class="pyvi…

In [21]:
ds_dgl.compute_aggregate_force(g, 1)

tensor([ 0.0015, -0.0008, -0.0011], device='cuda:0')

In [22]:
ds_dgl.compute_aggregate_force(g_pred,1)

tensor([ 0.0084,  0.0056, -0.1487], device='cuda:0')